Creates folium interactive maps of PBOT equity data and census population density

In [1]:
import folium
import pandas as pd
import geopandas as gpd
%matplotlib inline

In [3]:
tracts = gpd.read_file('../data/census_shapefiles/tl_2017_41_tract')
# Only plot Multnomah County
tracts = tracts[tracts['GEOID'].str.startswith('41051')]
gdf_wgs84 = tracts.to_crs({'init': 'epsg:4326'})

In [4]:
# Load PBOT equity matrix data
equity = pd.read_csv('../data/pbot_equity_matrix/pbot_equity_matrix.csv')
equity=equity[['FIPS', 'Combined']].rename(columns={'FIPS':'GEOID', 'Combined': 'Equity Score'})
equity['GEOID']=equity['GEOID'].astype(str)

# Load tract level population data
pop = pd.read_csv('../data/acs2017_pop_density/tract_population.csv')
pop['GEOID']=pop['state'].astype(str)+'0'+pop['county'].astype(str)+pop['tract'].astype(str).apply('{:0>6}'.format)

# Join data together on GEOIDs
merged = pd.merge(gdf_wgs84, equity, on='GEOID')
merged=pd.merge(merged, pop, on='GEOID')

In [8]:
# Calculate population density
merged['population_density'] = merged['population_estimate'] / merged['ALAND']
# Convert from people / meter^2 to people / mile^2
merged['population_density'] = merged['population_density'] * 2589988

In [13]:
m = folium.Map(location=[45.5236, -122.6750])
id_field='GEOID'
folium.Choropleth(geo_data = merged.to_json(), data = merged,
                columns = [id_field, 'Equity Score'], key_on = 'feature.properties.{}'.format(id_field),
                  fill_color = 'YlOrRd', legend_name='Equity Score', name='Equity Score'
                 ).add_to(m)

folium.Choropleth(geo_data = merged.to_json(), data = merged,
                columns = [id_field, 'population_density'], key_on = 'feature.properties.{}'.format(id_field),
                  fill_color = 'BuGn', legend_name='Population Density [people/square mile]', name='Population Density'
                 ).add_to(m)

folium.LayerControl().add_to(m)

m.save('../maps/population_density_and_equity_map.html')